In [0]:
# Load Silver table
silver_df = spark.read.table("retail_silver")

print("📌 Silver Table Schema:")
silver_df.printSchema()

display(silver_df.limit(5))


In [0]:
from pyspark.sql.functions import col, sum as _sum, date_trunc, round, to_date

# Aggregate daily revenue by country
daily_sales = silver_df.groupBy(
    col("Country"),
    to_date(date_trunc("day", col("InvoiceDate"))).alias("Date")   # clean date format
).agg(
    round(_sum(col("Quantity") * col("UnitPrice")), 2).alias("Revenue")  # round revenue
)

# Display first 40 rows ordered by date
display(daily_sales.orderBy("Date").limit(40))


In [0]:
daily_sales.write.format("delta") \
    .mode("overwrite") \
    .save("abfss://data@databricksmths.dfs.core.windows.net/retail/gold/daily_sales")


In [0]:
%sql
DROP TABLE IF EXISTS retail_gold_daily_sales;

CREATE TABLE retail_gold_daily_sales
USING DELTA
LOCATION 'abfss://data@databricksmths.dfs.core.windows.net/retail/gold/daily_sales';


In [0]:
gold_df = spark.read.table("retail_gold_daily_sales")

print(" Gold Daily Sales Schema:")
gold_df.printSchema()

display(gold_df.orderBy("Date").limit(20))


In [0]:
from pyspark.sql.functions import desc

top_products = silver_df.groupBy("ProductCode", "ProductDesc") \
    .agg(_sum("Quantity").alias("TotalQuantity")) \
    .orderBy(desc("TotalQuantity")) \
    .limit(10)

display(top_products)


In [0]:
top_products.write.format("delta") \
    .mode("overwrite") \
    .save("abfss://data@databricksmths.dfs.core.windows.net/retail/gold/top_products")


In [0]:
%sql
DROP TABLE IF EXISTS retail_gold_top_products;

CREATE TABLE retail_gold_top_products
USING DELTA
LOCATION 'abfss://data@databricksmths.dfs.core.windows.net/retail/gold/top_products';


In [0]:
%sql
SELECT * FROM retail_gold_top_products LIMIT 10;